In [1]:
import os
import json
from json import JSONDecodeError
import pandas as pd
from datetime import datetime
import re
import requests

data_path = os.path.join('D:/', 'data', 'reddit', 'wsb')

sub_end = "https://api.pushshift.io/reddit/search/submission/"
subcomment_end = "https://api.pushshift.io/reddit/submission/comment_ids/"
comment_end = "https://api.pushshift.io/reddit/search/comment/"

In [2]:
# load data - https://www.kaggle.com/mattpodolak/rwallstreetbets-posts-and-comments

filename_p = 'wallstreetbets_posts_06122020-06022021.csv.zip'
filename_p2 = 'reddit_wallstreetbets_06022021-08022021.json'
filename_p3 = 'reddit_wallstreetbets-gamestop_06022021-08022021.json'
filename_c = 'wallstreetbets_comments_06122020-06022021.csv.zip'

with open(os.path.join(data_path, filename_p2), 'r') as f:
    posts2 = json.load(f)
    
with open(os.path.join(data_path, filename_p3), 'r') as f:
    posts3 = json.load(f)

wsb_posts = pd.read_csv(os.path.join(data_path, filename_p))
wsb_posts2 = pd.DataFrame.from_records(posts2)
wsb_posts3 = pd.DataFrame.from_records(posts3)
wsb_comments = pd.read_csv(os.path.join(data_path, filename_c))

C:\programs\Anaconda3\envs\webscrape\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (4,81,82,83,84) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\programs\Anaconda3\envs\webscrape\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
wsb_comments.shape

(9559657, 37)

In [4]:
wsb_posts.head()

,all_awardings,allow_live_comments,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,...,secure_media_embed,banned_by,author_flair_template_id,author_cakeday,edited,gilded,distinguished,collections,crosspost_parent,crosspost_parent_list
0,[],False,readyrummy1,NaN,NaN,"[{'e': 'text', 't': '\x1b201202:4:1'}]",201202:4:1,dark,richtext,t2_7ppu74oe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Pluto_Muto,NaN,NaN,[],NaN,NaN,text,t2_9bo9tvhh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,Hiro08,NaN,NaN,[],NaN,NaN,text,t2_17panrff,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,uslashuname,NaN,NaN,[],NaN,NaN,text,t2_3nee5zpr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,dadwhomstfucks12,NaN,NaN,"[{'e': 'text', 't': '\x1b201202:3:1'}]",201202:3:1,dark,richtext,t2_50baj197,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
wsb_posts.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_text', 'author_flair_text_color',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'author_premium', 'awarders', 'can_mod_post', 'contest_mode',
       'created_utc', 'domain', 'full_link', 'gildings', 'id',
       'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_css_class',
       'link_flair_richtext', 'link_flair_template_id', 'link_flair_text',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler',
       'stickied', 'subreddit', 'subreddit

In [6]:
wsb_posts['created_utc'].head()

0    1607957430
1    1607957404
2    1607957384
3    1607957282
4    1607957280
Name: created_utc, dtype: int64

**Filtreringer:**

- Opslag med minimum 50 kommentarer og disses kommentarer
- 'GME' og/eller ‘Gamestop’ skal indgå i enten den ene eller begge af følgende:  
- En score på (up-/downvotes) min. 100 på opslag 
- For perioden 04/01-08/02 2021

Kolonner: comment_associated_award, comment_body, comment_created_utc, comment_id, comment_author_flair_richtext, comment_score, comment_post_id,
post_created_utc, post_id, post_is_meta, post_is_original_content, post_link_flair_richtext, post_num_comments, post_num_crossposts, post_score, post_selftext', post_title, post_upvote_ratio, post_gilded

In [7]:
import re

gme_re = re.compile(r'(.*\bgme\b.*)|(.*\bgamestop\b.*)', flags = re.DOTALL | re.IGNORECASE)

In [8]:
# Filter posts - GME

# Opslag med minimum 50 kommentarer
wsb_p_sub = wsb_posts.loc[wsb_posts['num_comments'].astype(float) > 50, :]

# En post score på 100 eller over
wsb_p_sub = wsb_p_sub.loc[wsb_p_sub['score'] >= 100, :]

# Fra perioden 4/1 til 8/2
filter_epoch_start = int(datetime(2021,1,4,0,0).timestamp())
filter_epoch_end = int(datetime(2021,2,8,0,0).timestamp())
wsb_p_sub = wsb_p_sub.loc[(wsb_p_sub['created_utc'] >= filter_epoch_start) & (wsb_p_sub['created_utc'] <= filter_epoch_end), :]

# ‘GME’ og/eller ‘Gamestop’ skal indgå
wsb_p_sub = wsb_p_sub.loc[wsb_p_sub['title'].str.contains(gme_re), :]

# fjerne dupletter
wsb_p_sub = wsb_p_sub.add_prefix('post_').drop_duplicates()

C:\programs\Anaconda3\envs\webscrape\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [9]:
wsb_p_sub.shape

(1163, 85)

In [10]:
def get_comments_from_data(postid, wsb_comments = wsb_comments, subcomment_end = subcomment_end):
    
    commentids = requests.get(f"{subcomment_end}{postid}").json().get('data')
    
    wsb_post_c = wsb_comments[wsb_comments['id'].astype(str).isin(commentids)]
    wsb_post_c['post_id'] = wsb_post_c['link_id'].apply(lambda postid: re.sub(".*_", "", postid))
    wsb_post_c = wsb_post_c.reset_index(drop = True).add_prefix('comment_')
    
    return(wsb_post_c)

In [11]:
post_ids = list(wsb_p_sub['post_id'])
wsb_subset = pd.DataFrame()
failed_ids = []
#post_ids = post_ids[post_ids.index(post_id):len(post_ids)]

for c, post_id in enumerate(post_ids, start = 1):
    try:
        wsb_pcs = get_comments_from_data(post_id)
    
        wsb_m = pd.merge(wsb_pcs, wsb_p_sub, how = 'left', left_on = 'comment_post_id', right_on = 'post_id')
    
        wsb_subset = wsb_subset.append(wsb_m, ignore_index = True)
    
    except JSONDecodeError:
        failed_ids.append(post_id)
    
    progress = "|{0}| {1:.2f} %".format(("="*int(c/len(post_ids) * 50)).ljust(50), c/len(post_ids) * 100)
    print(progress, end = "\r")

C:\programs\Anaconda3\envs\webscrape\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
wsb_subset.shape

(719419, 123)

In [13]:
# Add data 6/2-8/2
wsb_posts2 = wsb_posts2.explode('comments').reset_index(drop=True).add_prefix('post_')
wsb_posts2 = pd.merge(wsb_posts2, pd.json_normalize(wsb_posts2['post_comments']).add_prefix('comment_'), left_index=True, right_index=True)

wsb_posts3 = wsb_posts3.explode('comments').reset_index(drop=True).add_prefix('post_')
wsb_posts3 = pd.merge(wsb_posts3, pd.json_normalize(wsb_posts3['post_comments']).add_prefix('comment_'), left_index=True, right_index=True)

wsb_subset_final = wsb_subset.append(wsb_posts2)
wsb_subset_final = wsb_subset_final.append(wsb_posts3)

cols_keep = ['comment_associated_award', 'comment_body', 'comment_created_utc', 'comment_id', 'comment_author_flair_richtext', 'comment_score', 'comment_post_id',
             'post_created_utc', 'post_id', 'post_is_meta', 'post_is_original_content', 'post_link_flair_richtext', 'post_num_comments', 'post_num_crossposts', 
             'post_score', 'post_selftext', 'post_title', 'post_upvote_ratio', 'post_gilded']

In [14]:
wsb_subset_final = wsb_subset_final.loc[:, cols_keep].drop_duplicates(subset = ['post_id', 'comment_id', 'post_score', 'comment_score', 'comment_created_utc', 
                                                                               'post_created_utc'])

In [15]:
outname = "reddit_wallstreetbets_gme_04012021-08022021.zip"

wsb_subset_final.to_csv(os.path.join(data_path, outname), index = False, compression = "zip")

In [111]:
wsb_subset_filt = wsb_subset.loc[wsb_subset['comment_body'].str.len() > 20, :]

In [112]:
wsb_subset_filt.shape

(334008, 123)

In [114]:
wsb_subset_sample = wsb_subset.sample(10000)

In [115]:
outname = "reddit_wallstreetbets_gme_01012021-20022021_sample.zip"

wsb_subset_sample.to_csv(os.path.join(data_path, outname), index = False, compression = "zip")

In [ ]:
# Filter comments - GME

# filter comments
wsb_c_sub = wsb_comments.loc[wsb_comments['body'].str.contains(gme_re), 'link_id']
wsb_c_sub['post_id'] = wsb_c_sub['link_id'].apply(lambda postid: re.sub(".*_", "", postid))

# filter post ids
wsb_c_filt = wsb_c_sub.loc[~wsb_c_sub['post_id'].isin(post_ids), :]

# filter post data
new_pids = list(wsb_c_filt['post_id'].unique())
wsp_cp_sub = wsb_posts.loc[wsb_posts['id'].isin(new_pids), :]

# Opslag med minimum 50 kommentarer
wsp_cp_sub = wsp_cp_sub.loc[wsp_cp_sub['num_comments'].astype(float) > 50, :]

# Enten en score på (up-/downvotes) under 100 eller over 100 på opslag
wsp_cp_sub = wsp_cp_sub.loc[(wsp_cp_sub['score'] <= -100) | (wsp_cp_sub['score'] >= 100), :]

# Efter 1/1-2021
filter_epoch = int(datetime(2021,1,1,0,0).timestamp())
wsp_cp_sub = wsp_cp_sub.loc[wsp_cp_sub['created_utc'] >= filter_epoch, :]

# add post data
wsb_c_filt = wsb_c_filt.add_prefix('comment_')
wsp_cp_sub = wsp_cp_sub.add_prefix('post_')
wsb_cp = pd.merge(wsb_c_filt, wsp_cp_sub, how = 'left', left_on = 'comment_post_id', right_on = 'post_id')

# add to data
#wsb_subset_all = wsb_subset.append(wsb_cp, ignore_index = True)